In [0]:
%run ./contract_first_helpers

In [0]:
import os
import json
import yaml
from datacontract.data_contract import DataContract

In [0]:
dbutils.widgets.removeAll()

dbutils.widgets.text("contracts_location", "/Volumes/flightstats_historical_dev_azr_westus/data_contract_test/contracts/")
dbutils.widgets.text("contract_json_file", "flight_stats.json")
dbutils.widgets.text("odcs_template_file", "odcs_template.yml")

contracts_location = dbutils.widgets.get("contracts_location")
contract_json_file = dbutils.widgets.get("contract_json_file")
odcs_template_file = dbutils.widgets.get("odcs_template_file")



In [0]:
host = spark.conf.get("spark.databricks.workspaceUrl")
environment = "development"

In [0]:
contract_json_path = f"{contracts_location}{contract_json_file}"
odcs_template_path = f"{contracts_location}{odcs_template_file}"

filename_with_extension = os.path.basename(contract_json_file)
filename_without_extension = os.path.splitext(filename_with_extension)[0]

odcs_yaml_path = f"{contracts_location}/{filename_without_extension}.yml"

In [0]:
def create_odcs_yaml(odcs, yaml_file):
    # Convert the provided odcs object to a YAML string representation
    yaml_string = yaml.dump(odcs, default_flow_style=False)
    # Write the YAML representation to the specified file
    with open(yaml_file, 'w') as file:
        yaml.dump(odcs, file, default_flow_style=False)

In [0]:

odcs = convert_json_to_odcs(contract_json_path, odcs_template_path, host, environment)


In [0]:
create_odcs_yaml(odcs, odcs_yaml_path)

In [0]:
data_contract = DataContract(data_contract_file=odcs_yaml_path, spark=spark)

result = []
for check in data_contract.lint().checks:
    if check.reason is None:
        continue
    result.append({"reason": check.reason, "result": str(check.result)})

spark.createDataFrame(result).display()
